In [47]:
import numpy as np
from libs.global_variable import ROOT_DIR

path = ROOT_DIR + 'Data/BuFF/buff_release_rot_const/sequences/00096/shortshort_hips/shortshort_hips_000010_cano.npy'

cano = np.load(path, allow_pickle=True).item()
cano_verts = cano['cano_points']
cano_normals = cano['cano_normals']
valid_mask = cano['valid_mask']

path_scan = ROOT_DIR + 'Data/BuFF/buff_release_rot_const/sequences/00096/shortshort_hips/shortshort_hips_000010.npy'

scan = np.load(path_scan, allow_pickle=True).item()
scan_verts = scan['points_posed_cloth']
scan_normals = scan['normals_posed_cloth']

In [51]:
def _get_repeated_idx(num_points_input, num_points):

    assert(num_points_input <= num_points)
    idx_org = np.arange(0, num_points_input)

    num_points_rest = num_points - num_points_input
    if num_points_rest <= num_points_input:
        idx_rest = np.random.choice(np.arange(0, num_points_input), num_points_rest, replace=False)
    elif num_points_rest > num_points_input:
        num_points_rest_1 = num_points_input
        idx_rest_1 = np.random.choice(np.arange(0, num_points_input), num_points_rest_1, replace=False)
        num_points_rest_2 = num_points_rest - num_points_rest_1
        idx_rest_2 = np.random.choice(np.arange(0, num_points_input), num_points_rest_2, replace=False)
        idx_rest = np.concatenate((idx_rest_1, idx_rest_2))

    idx_list = np.concatenate((idx_org, idx_rest))

    return idx_list

idx_list = _get_repeated_idx(cano_verts.shape[0], 30000)

In [48]:
valid_mask.shape

(24169,)

In [55]:
cano_verts[idx_list].shape

(30000, 3)

In [53]:
scan_verts.transpose()[valid_mask][idx_list].shape

(30000, 3)

In [2]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(cano_verts)
pcd.normals = o3d.utility.Vector3dVector(cano_normals)

# write point cloud
o3d.io.write_point_cloud("visualization/00096_shortlong_hips_000216_cano.ply", pcd)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


True

In [1]:
from dataloaders.dataloader_buff import DataLoader_Buff_depth

splt_file = '/home/yuxuan/project/NeuralSurfaceField/assets/data_split/buff_female_train_val.pkl' 
subject_index_dict = {}
subject_index_dict.update({"00032_shortlong": 0,
                            "00096_shortlong": 1})
train_dataset = DataLoader_Buff_depth(mode='train', batch_size=2, num_workers=4, split_file=splt_file, subject_index_dict=subject_index_dict)
train_loader = train_dataset.get_loader()


Loading train data...


100%|██████████| 569/569 [00:04<00:00, 137.15it/s]

Data loaded, in total 569 train examples.



In [2]:
for i, batch in enumerate(train_loader):
    break

In [9]:
import open3d as o3d

# write function which uses open3d to write point cloud
def write_pcd(path, points, normals):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.normals = o3d.utility.Vector3dVector(normals)
    o3d.io.write_point_cloud(path, pcd)

path_cano = '/home/yuxuan/project/NeuralSurfaceField/visualization/cano.ply'
path_posed = '/home/yuxuan/project/NeuralSurfaceField/visualization/posed.ply'

# write cano point cloud
write_pcd(path_cano, batch['cano_points'][0].cpu().numpy(), batch['cano_normals'][0].cpu().numpy())
# write posed point cloud
write_pcd(path_posed, batch['scan_points'][0].cpu().numpy().transpose(), batch['scan_normals'][0].cpu().numpy().transpose())

In [5]:
import open3d as o3d
import numpy as np

dict_ = np.load('/home/yuxuan/project/NeuralSurfaceField/Data_scan/BuFF/buff_release/sequences/00159/shortshort_twist_tilt_left/shortshort_twist_tilt_left.000050.npz')

# o3d point cloud visualization
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(dict_['v_posed'])
o3d.visualization.draw_geometries([pcd])

In [11]:
import pickle as pkl

split_file = '/mnt/qb/work/ponsmoll/yxue80/project/NeuralSurfaceField/assets/data_split/buff_female_train_val.pkl'

with open(split_file, "rb") as f:
    split = pkl.load(f)

In [10]:
from libs.global_variable import ROOT_DIR
import os

split_org = {
    'train': [],
    'val': []
}

for subj_train in split['train']:
    split_org['train'].append(subj_train.replace('Data_male', 'Data'))

for subj_val in split['val']:
    split_org['val'].append(subj_val.replace('Data_male', 'Data'))


pkl.dump(split_org, open(split_file, 'wb'))

In [3]:
batch['cano_points'].shape

torch.Size([2, 30000, 3])

In [3]:
import os
import torch

# load old checkpoint
shapefusion_path = '/mnt/qb/work/ponsmoll/yxue80/project/shapefusion/experiments/PoseImplicit_exp_id_808'
shapefusion_checkpoint_path = os.path.join(shapefusion_path, 'checkpoints', 'checkpoint_epoch_2150.tar')

print('Loaded checkpoint from: {}'.format(shapefusion_checkpoint_path))
checkpoint = torch.load(shapefusion_checkpoint_path)

print(checkpoint.keys())

# save to new checkpoint
path = 'checkpoint_epoch_{}.tar'.format(checkpoint['epoch'])
if not os.path.exists(path):
    model_weights = {'epoch': checkpoint['epoch'],
                    'optimizer_state_dict': checkpoint['optimizer_state_dict'], 
                    'scheduler_state_dict': checkpoint['scheduler_state_dict'],
                    'feat_optimizer_state_dict': checkpoint['feat_optimizer_state_dict'],
                    'feat_scheduler_state_dict': checkpoint['feat_scheduler_state_dict'],
                    'pose_encoder_state_dict': checkpoint['pose_encoder_state_dict'],
                    'nsf_decoder_state_dict': checkpoint['shape_geometry_decoder_state_dict']}
    torch.save(model_weights, path)

Loaded checkpoint from: /mnt/qb/work/ponsmoll/yxue80/project/shapefusion/experiments/PoseImplicit_exp_id_808/checkpoints/checkpoint_epoch_2150.tar
dict_keys(['epoch', 'optimizer_state_dict', 'scheduler_state_dict', 'feat_optimizer_state_dict', 'feat_scheduler_state_dict', 'pose_encoder_state_dict', 'shape_geometry_decoder_state_dict', 'conditional_ndf_state_dict'])


In [7]:
import torch

# Create a tensor with requires_grad enabled
tensor = torch.tensor([1, 2, 3], dtype=torch.float32, requires_grad=False)
tensor2 = torch.tensor([1, 2, 4], dtype=torch.float32, requires_grad=False)

mask = tensor > 2

tensor[mask] = tensor2[mask]
tensor

tensor([1., 2., 4.])

In [3]:
import numpy as np
import os

path = 'data_animation/aist_demo/seqs'

pose_list = []
transl_list = []

for file in sorted(os.listdir(path)):
    print(file)
    file_path = os.path.join(path, file)
    # Load data
    data = dict(np.load(file_path))
    pose_list.append(data['pose'])
    transl_list.append(data['transl'])

summed_file = {
    'pose': pose_list,
    'transl': transl_list
}

np.save('data_animation/aist_poses.npy', summed_file)

00000.npz
00002.npz
00004.npz
00006.npz
00008.npz
00010.npz
00012.npz
00014.npz
00016.npz
00018.npz
00020.npz
00022.npz
00024.npz
00026.npz
00028.npz
00030.npz
00032.npz
00034.npz
00036.npz
00038.npz
00040.npz
00042.npz
00044.npz
00046.npz
00048.npz
00050.npz
00052.npz
00054.npz
00056.npz
00058.npz
00060.npz
00062.npz
00064.npz
00066.npz
00068.npz
00070.npz
00072.npz
00074.npz
00076.npz
00078.npz
00080.npz
00082.npz
00084.npz
00086.npz
00088.npz
00090.npz
00092.npz
00094.npz
00096.npz
00098.npz
00100.npz
00102.npz
00104.npz
00106.npz
00108.npz
00110.npz
00112.npz
00114.npz
00116.npz
00118.npz
00120.npz
00122.npz
00124.npz
00126.npz
00128.npz
00130.npz
00132.npz
00134.npz
00136.npz
00138.npz
00140.npz
00142.npz
00144.npz
00146.npz
00148.npz
00150.npz
00152.npz
00154.npz
00156.npz
00158.npz
00160.npz
00162.npz
00164.npz
00166.npz
00168.npz
00170.npz
00172.npz
00174.npz
00176.npz
00178.npz
00180.npz
00182.npz
00184.npz
00186.npz
00188.npz
00190.npz
00192.npz
00194.npz
00196.npz
00198.npz


In [17]:
import numpy as np
frame_0 = np.load('data_animation/aist_poses.npy', allow_pickle=True).item()['pose'][0]
frame_5 = np.load('data_animation/aist_poses.npy', allow_pickle=True).item()['pose'][5]
frame_10 = np.load('data_animation/aist_poses.npy', allow_pickle=True).item()['pose'][10]
frame_12 = np.load('data_animation/aist_poses.npy', allow_pickle=True).item()['pose'][12]
frame_13 = np.load('data_animation/aist_poses.npy', allow_pickle=True).item()['pose'][13]
frame_14 = np.load('data_animation/aist_poses.npy', allow_pickle=True).item()['pose'][14]


In [23]:
from libs.smpl_paths import SmplPaths
# unposed and unshaped smpl mesh (reference)
ref_sp = SmplPaths(gender='male')
ref_smpl = ref_sp.get_smpl()
ref_smpl.pose[:], ref_smpl.betas[:], ref_smpl.trans[:] = frame_0, 0, 0
smpl_skinning = ref_smpl.weights.r
smpl_shape = ref_smpl.r
smpl_shape.setflags(write=1)

In [24]:
import open3d as o3d

# save point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(smpl_shape)
o3d.io.write_point_cloud("visualization/smpl_shape.ply", pcd)

True